# Type annotations

Python has an option to annotate types, which is useful during relatively serious development.

Check resources:

- [Python type checking](https://realpython.com/python-type-checking/#type-systems) article on real python.
- [PEP 484](https://peps.python.org/pep-0484/) that introduce type hints.
- [PEP 526](https://peps.python.org/pep-0526/) that introduce variable annotations.

In [ ]:
import sys
sys.path.append("/tmp")

## Usecases

In general, you can define types of parameters and return values for the function and specify the type for some variable/attribute. This doesn't affect the behavior of the interpreter in any way, but different code analisys tools make use of it.

---

The following cell creates python file that defines function that expect to take `float` and `int` and return `bool`. The function is wrapped with `typeguard.typechecked` decorator, which throws raise an error every time it is called with wrong types.

In [24]:
%%writefile /tmp/typed_fun.py
from typeguard import typechecked

@typechecked
def some_function(arg1: float, arg2: int) -> bool:
    return arg1 > arg2

Overwriting /tmp/typed_fun.py


The following code shows the exception you get when you try to pass parameters of the wrong type to the function. 

In [37]:
from typed_fun import some_function
try: some_function(5.5, 3.3) 
except Exception as e: print(e)

argument "arg2" (float) is not an instance of int


The next cell generates a file that contains a variable that shoudl be of the datatype `float` datatype, but assigns `str` to the variable.

In [21]:
%%writefile /tmp/typed_var.py
pi: float = "test"

Overwriting /tmp/typed_var.py


`mypy` static analisator throws corresponding error to that file.

In [22]:
!python3 -m mypy /tmp/typed_var.py

/tmp/typed_var.py:1: error: Incompatible types in assignment (expression has type "str", variable has type "float")  [assignment]
Found 1 error in 1 file (checked 1 source file)


## Annotation cases

This section provided an overview of the various syntax and `typing` attributes that can be used to define types.

| **Category**         | **Syntax**                               | **Description**                                         | **Example**                                 |
|----------------------|-------------------------------------------|---------------------------------------------------------|----------------------------------------------|
| **Basic types**      | `int`, `str`, `bool`, `float`, `None`    | Built-in types                                          | `x: int = 5`                                 |
| **Union**            | `typing.Union[A, B]` / `A \| B` (3.10+)    | One of several types                                    | `x: int \| None` or `x: Union[int, None]`      |
| **Optional**         | `typing.Optional[T]`                             | Shorthand for `Union[T, None]`                          | `x: Optional[str] = None`                    |
| **List**             | `typing.List[T]` / `list[T]` (3.9+)       | A list of items                                         | `names: list[str]`                           |
| **Tuple**            | `typing.Tuple[<types>]` / tuple            | A fixed-size tuple                                      | `point: Tuple[int, int]`                    |
| **Dict**             | `typing.Dict[K, V]` / `dict[K, V]` (3.9+)        | Dictionary with keys and values                        | `user: dict[str, int]`                       |
| **Set**              | `typing.Set[T]` / `set[T]`                       | A set of items                                          | `tags: set[str]`                             |
| **Callable**         | `typing.Callable[[Args], Return]`                | A function signature                                    | `f: Callable[[int, str], bool]`             |
| **Literal**          | `typing.Literal[val1, val2]`                     | Restrict to specific literal values                     | `mode: Literal["r", "w"]`                    |
| **Any**              | `typing.Any`                                     | Skip type checking                                      | `data: Any`                                  |
| **None**             | `None`                                           | Variable takes `None` value                             | `def void() -> None`                         |
| **NoReturn**         | `typing.NoReturn`                                | Function never returns                                  | `def fail() -> NoReturn:`                    |
| **Annotated**        | `typing.Annotated[T, metadata]`                  | Attach metadata to a type                               | `x: Annotated[int, "positive"]`              |
| **NewType**          | `typing.NewType('Name', BaseType)`               | Create a distinct type based on an existing one         | `UserId = NewType('UserId', int)`            |
| **TypeVar**          | `typing.TypeVar('T')`                            | Generic type variable                                   | `def f(x: T) -> T:`                          |
| **Generic**          | `typing.Generic[T]`                              | Base class for generic classes                          | `class Box(Generic[T]):`                     |
| **Self**             | `typing.Self` (3.11+)                     | Return type of method returning its own class           | `def clone(self) -> Self:`                   |
| **ClassVar**         | `typing.ClassVar[T]`                             | Type of class-level variables                           | `version: ClassVar[str] = "1.0"`             |
| **Final**            | `typing.Final`                                   | Marks values as constants                               | `MAX_SIZE: Final[int] = 100`                 |
| **TypedDict**        | `class Movie(typing.TypedDict):`                | Dict with specific key types                            | `title: str; year: int`                      |
| **Protocol**         | `class P(typing.Protocol):`                      | Structural subtyping (duck typing)                      | `def read(self) -> bytes:`                   |
| **Union with ellipsis** | `tuple[int, ...]`                     | Homogeneous tuple of any length                         | `values: Tuple[int, ...]`                    |

Check more accurate description with examples on the [conrresponding page](type_annotations/annotation_cases.ipynb).

## Type aliases

Note that you can save your annotations as regular python object - your programmes will be neater.

---

Below is an example that defines a function to create triangles as a tuple of three two-dimensional points. To make things simplier, you can define an annotation for the point and use it to define an annotation for the triangle.

In [5]:
from random import random

# Here is straighforward definition
def get_triangle() -> tuple[
        tuple[float, float],
        tuple[float, float],
        tuple[float, float]
    ]:
    return (
        (random(), random())
        for i in range(3)
    )

# Same but shorter
point = tuple[float, float]
def get_triangle() -> tuple[point, point, point]:
    return (
        (random(), random())
        for i in range(3)
    )

## `__annotations__` attribute

The `__annotations__` attribute allows you to retrieve annotated types for a Python object.

---

For function it returns a dictionary with keys corresponding to the names of the arguments and values corresponding to the types of the arguments. The return type of the function can be accessed using the `return` key.

In [1]:
def some_function(arg1: float, arg2: bool) -> int:
    if arg1 > 10 and arg2:
        return 20

some_function.__annotations__

{'arg1': float, 'arg2': bool, 'return': int}

You can simply access it from any Python namespace to get annotations of the waribles in that namespace.

In [2]:
new_variable : float = 10
test_variable : float = 10

__annotations__

{'new_variable': float, 'test_variable': float}

## Mypy type checker

Tool that allows you to check if the programme respects the specified type hints.

---

So in the following example a function is defined that takes two arguments and they are annotated as `int`. And just in the next line funciton call just ignores annotations.

In [8]:
%%writefile type_annotations_files/mypy_example.py
def bin_sum(a: int, b: int):
    return a+b

print(bin_sum("a", "b"))

Overwriting type_annotations_files/mypy_example.py


Just execution of the program won't cause any problems.

In [9]:
!python3 type_annotations_files/mypy_example.py

ab


Which strings do not meet the type annotation.

In [10]:
!python3 -m mypy type_annotations_files/mypy_example.py

type_annotations_files/mypy_example.py:4: error: Argument 1 to "bin_sum" has incompatible type "str"; expected "int"  [arg-type]
type_annotations_files/mypy_example.py:4: error: Argument 2 to "bin_sum" has incompatible type "str"; expected "int"  [arg-type]
Found 2 errors in 1 file (checked 1 source file)


### Get mypy types

`Mypy` has it's own type detection system, so sometimes it's useful to know how to check what type it's detecting. You can do this using the `reveal_type` and `reveal_locals` functions.

**Note** You don't need to import them or something `mypy` will understand them by itself.

Here is an example of the `reveal_type` function, which allows you to understand how mypy interprets the type of a given name.

In [8]:
%%writefile type_annotations_files/mypy_reveal_type.py
import numpy as np
reveal_type(np.pi)

Writing type_annotations_files/mypy_reveal_type.py


In [10]:
!python3 -m mypy type_annotations_files/mypy_reveal_type.py

type_annotations_files/mypy_reveal_type.py:2: note: Revealed type is "builtins.float"
Success: no issues found in 1 source file


And an example of `reveal_locals`, which prints out information about all the variables in the namespace.

In [11]:
%%writefile type_annotations_files/mypy_reveal_locals.py
my_float = 10.
my_int = 10

reveal_locals()

Writing type_annotations_files/mypy_reveal_locals.py


In [12]:
!python3 -m mypy type_annotations_files/mypy_reveal_locals.py

type_annotations_files/mypy_reveal_locals.py:4: note: Revealed local types are:
type_annotations_files/mypy_reveal_locals.py:4: note:     my_float: builtins.float
type_annotations_files/mypy_reveal_locals.py:4: note:     my_int: builtins.int
Success: no issues found in 1 source file


### Wrong syntax

The Python interpreter doesn't throw exceptions even if there are annotations that are defined incorrectly. But `mypy` will throw errors on it's side.

Here is an example where we are trying to annotate the dictionary, but instead of annotating key and value types, we have only annotated one.

In [7]:
%%writefile type_annotations_files/mypy_wrong_syntax.py
my_dict: dict[int]

Writing type_annotations_files/mypy_wrong_syntax.py


If you try to run it just with the python interpreter, it works fine.

In [8]:
!python3 type_annotations_files/mypy_wrong_syntax.py

But `mypy` raises corresponding error.

In [9]:
!python3 -m mypy type_annotations_files/mypy_wrong_syntax.py

type_annotations_files/mypy_wrong_syntax.py:1: error: "dict" expects 2 type arguments, but 1 given  [type-arg]
Found 1 error in 1 file (checked 1 source file)
